**QUE- 6 Implementing Neural Style Transfer**

In [1]:
import os
import torch
import numpy as np
from torch.optim import Adam, LBFGS
from torch.autograd import Variable
import torchvision.models as models
import cv2 as cv
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
import sys
sys.path.append('/kaggle/input/neural-style-transfer-data')

In [2]:
Imagenet_mean = [123.675, 116.28, 103.53]
Imagenet_std = [1, 1, 1]

def load_image(img_path, target_shape=None):
    if not os.path.exists(img_path):
        raise Exception(f'Path does not exist: {img_path}')
    img = cv.imread(img_path)[:, :, ::-1]  

    if target_shape is not None:  
        if isinstance(target_shape, int) and target_shape != -1:  
            current_height, current_width = img.shape[:2]
            new_height = target_shape
            new_width = int(current_width * (new_height / current_height))
            img = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_CUBIC)
        else:  
            img = cv.resize(img, (target_shape[1], target_shape[0]), interpolation=cv.INTER_CUBIC)

    img = img.astype(np.float32)  
    img /= 255.0 
    return img

def prepare_img(img_path, target_shape, device):
    img = load_image(img_path, target_shape=target_shape)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255)),
        transforms.Normalize(mean=Imagenet_mean, std=Imagenet_std)
    ])

    img = transform(img).to(device).unsqueeze(0)

    return img

def save_image(img, img_path):
    if len(img.shape) == 2:
        img = np.stack((img,) * 3, axis=-1)
    cv.imwrite(img_path, img[:, :, ::-1])  

def generate_out_name(config):
    prefix = os.path.basename(config['content_img_name']).split('.')[0] + '_' + os.path.basename(config['style_img_name']).split('.')[0]
    if 'reconstruct_script' in config:
        suffix = f'_o_{config["optimizer"]}_h_{str(config["height"])}_m_{config["model"]}{config["img_format"][1]}'
    else:
        suffix = f'_o_{config["optimizer"]}_i_{config["init_method"]}_h_{str(config["height"])}_m_{config["model"]}_cw_{config["content_weight"]}_sw_{config["style_weight"]}_tv_{config["tv_weight"]}{config["img_format"][1]}'
    return prefix + suffix

def save_and_maybe_display(opt_img, dump_path, config, img_id, no_iter, display=False):
    savefreq = config['savefreq']
    out_img = opt_img.squeeze(axis=0).to('cpu').detach().numpy()
    out_img = np.moveaxis(out_img, 0, 2)

    if img_id == no_iter-1 or (savefreq > 0 and img_id % savefreq == 0):
        img_format = config['img_format']
        out_name = str(img_id).zfill(img_format[0]) + img_format[1] if savefreq != -1 else generate_out_name(config)
        dump_img = np.copy(out_img)
        dump_img += np.array(Imagenet_mean).reshape((1, 1, 3))
        dump_img = np.clip(dump_img, 0, 255).astype('uint8')
        cv.imwrite(os.path.join(dump_path, out_name), dump_img[:, :, ::-1])

    if display:
        plt.imshow(np.uint8(uint8_range(out_img)))
        plt.show()

def uint8_range(x):
    if isinstance(x, np.ndarray):
        x -= np.min(x)
        x /= np.max(x)
        x *= 255
        return x
    else:
        raise ValueError(f'Expected numpy array got {type(x)}')

In [3]:
class Vgg19(torch.nn.Module):
    def __init__(self, requires_grad=False, show_progress=False, use_relu=True):
        super().__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True, progress=show_progress).features
        if use_relu: 
            self.layer_names = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'conv4_2', 'relu5_1']
            self.offset = 1
        else:
            self.layer_names = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv4_2', 'conv5_1']
            self.offset = 0
        self.contfeatmap_ind = 4  
        
        self.stylefeatmap_ind = list(range(len(self.layer_names)))
        self.stylefeatmap_ind.remove(4)  

        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.slice6 = torch.nn.Sequential()
        for x in range(1+self.offset):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(1+self.offset, 6+self.offset):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(6+self.offset, 11+self.offset):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(11+self.offset, 20+self.offset):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(20+self.offset, 22):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        for x in range(22, 29++self.offset):
            self.slice6.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, x):
        x = self.slice1(x)
        layer1_1 = x
        x = self.slice2(x)
        layer2_1 = x
        x = self.slice3(x)
        layer3_1 = x
        x = self.slice4(x)
        layer4_1 = x
        x = self.slice5(x)
        conv4_2 = x
        x = self.slice6(x)
        layer5_1 = x
        vgg_outputs = namedtuple("VggOutputs", self.layer_names)
        out = vgg_outputs(layer1_1, layer2_1, layer3_1, layer4_1, conv4_2, layer5_1)
        return out

In [4]:
def prepare_model(model, device):
    model = Vgg19(requires_grad=False, show_progress=True)
    
    contfeatmap_ind = model.contfeatmap_ind
    stylefeatmap_ind = model.stylefeatmap_ind
    layer_names = model.layer_names

    cont_indname = (contfeatmap_ind, layer_names[contfeatmap_ind])
    style_fms_indices_names = (stylefeatmap_ind, layer_names)
    return model.to(device).eval(), cont_indname, style_fms_indices_names

In [5]:
def gram_matrix(x, should_normalize=True):
    (b, ch, h, w) = x.size()
    features = x.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t)
    if should_normalize:
        gram /= ch * h * w
    return gram

def total_variation(y):
    return torch.sum(torch.abs(y[:, :, :, :-1] - y[:, :, :, 1:])) + \
           torch.sum(torch.abs(y[:, :, :-1, :] - y[:, :, 1:, :]))

def compute_losses(model, img, target_rep, content_idx, style_indices, config):
    target_content, target_styles = target_rep
    features = model(img)

    if isinstance(features, tuple):
        features = list(features)

    content_loss = torch.nn.functional.mse_loss(
        target_content, features[content_idx].squeeze(0)
    )

    style_loss = sum(
        torch.nn.functional.mse_loss(gram_matrix(features[idx]), gs)
        for idx, gs in zip(style_indices, target_styles)
    ) / len(target_styles)

    tv_loss = total_variation(img)

    total_loss = (
        config['content_weight'] * content_loss +
        config['style_weight'] * style_loss +
        config['tv_weight'] * tv_loss
    )

    return total_loss, content_loss, style_loss, tv_loss

In [6]:
def optimize_image(model, img, target_rep, content_idx, style_indices, config):
    optimizer_type = config['optimizer']
    num_iter = {'adam': 3000, 'lbfgs': 1000}[optimizer_type]

    if optimizer_type == 'adam':
        optimizer = Adam([img], lr=10)
        for i in range(num_iter):
            optimizer.zero_grad()
            losses = compute_losses(
                model, img, target_rep, content_idx, style_indices, config
            )
            losses[0].backward()
            optimizer.step()
            
            # save_and_maybe_display(img, config['output_img_dir'], config, i, num_iter, False)
            # print(f"Adam | Iter {i+1}/{num_iter}: Loss={losses[0].item():.4f}")
            
    elif optimizer_type == 'lbfgs':
        optimizer = LBFGS([img], max_iter=num_iter, line_search_fn='strong_wolfe')
        # iter_count = 0

        def closure():
            # nonlocal iter_count
            optimizer.zero_grad()
            losses = compute_losses(
                model, img, target_rep, content_idx, style_indices, config
            )
            losses[0].backward()
            # iter_count += 1
            
            # save_and_maybe_display(img, config['output_img_dir'], config, iter_count, num_iter, False)
            # print(f"L-BFGS | Iter {iter_count}/{num_iter}: Loss={losses[0].item():.4f}")
            return losses[0]

        optimizer.step(closure)
        
    return img

In [7]:
def neural_style_transfer(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    content_img = prepare_img(config['content_img_name'], config['height'], device)
    style_img = prepare_img(config['style_img_name'], config['height'], device)

    model, cont_indname, style_fms_indices_names = prepare_model(config['model'], device)

    content_features = model(content_img)[cont_indname[0]].squeeze(0).detach()
    style_features = [
        gram_matrix(model(style_img)[idx].detach())
        for idx in style_fms_indices_names[0]
    ]

    opt_img = Variable(
        content_img.clone(), requires_grad=True
    ) if config['init_method'] == 'content' else Variable(
        style_img.clone(), requires_grad=True
    )

    optimized_img = optimize_image(
        model,
        opt_img,
        (content_features, style_features),
        cont_indname[0],
        style_fms_indices_names[0],
        config
    )
    
    final_img_path = os.path.join(config['output_img_dir'], generate_out_name(config))
    save_image(optimized_img.squeeze(0).to('cpu').detach().numpy().transpose(1, 2, 0), final_img_path)
    print(f"Final image saved to {final_img_path}")

In [8]:
if __name__ == "__main__":
    config = {
        'content_img_name': '/kaggle/input/neural-style-transfer-data/data/content-images/lion.jpg',
        'style_img_name': '/kaggle/input/neural-style-transfer-data/data/style-images/vg_wheat_field.jpg',
        'height': 512,  
        'model': 'vgg19',  
        'init_method': 'content',  
        'optimizer': 'lbfgs',  
        'content_weight': 1e5,
        'style_weight': 1e8,
        'tv_weight': 1e-4,
        'output_img_dir': 'output/',  
        'img_format': (5, '.png'),  
        'savefreq': 50  
    }
    os.makedirs(config['output_img_dir'], exist_ok=True)
    neural_style_transfer(config)
    print('done')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 215MB/s] 


Final image saved to output/lion_vg_wheat_field_o_lbfgs_i_content_h_512_m_vgg19_cw_100000.0_sw_100000000.0_tv_0.0001.png
done


**Conclusion: 
Neural Style Transfer (NST) using VGG19 leverages the pre-trained convolutional layers of the network to extract and manipulate image features, effectively blending the content of one image with the artistic style of another. VGG19, known for its deep architecture and strong feature extraction capabilities, captures content information from deeper layers and style information from multiple shallower layers. By optimizing a loss function that balances content and style reconstruction, NST iteratively updates an input image to achieve the desired artistic transformation. Despite challenges like high computational costs and hyperparameter tuning, VGG19’s robust performance makes it a popular choice for NST, enabling creative applications in art and design.**

In [9]:
# import shutil

# shutil.rmtree('/kaggle/working/output')